In [1]:
import pandas as pd
import requests
import json
import logging
from bs4 import BeautifulSoup
import time

In [2]:
# GET TOP TOURISM COUNTRIES
df_tourism = pd.read_csv('./data/most_traveled_countries.csv')
# SOURCE: http://worldpopulationreview.com/countries/most-visited-countries/
# df_tourism.head()

top20 = list(df_tourism.sort_values(by='touristArrivals', ascending=False)['name'][:20])
print(top20)

['France', 'United States', 'Spain', 'China', 'Italy', 'United Kingdom', 'Germany', 'Mexico', 'Thailand', 'Turkey', 'Austria', 'Malaysia', 'Hong Kong', 'Greece', 'Russia', 'Japan', 'Canada', 'Portugal', 'Saudi Arabia', 'Poland']


In [3]:
# GET COUNTRIES
country_names = [c_name for c_name in pd.read_csv('./data/country_names.csv')['name']]
# SOURCES: http://worldpopulationreview.com/countries/most-visited-countries/
    # & https://www.travel-advisory.info/api


In [4]:
len(country_names)

178

In [5]:
# GET ALL URLS FOR 20 COUNTRIES
articles = []
for country in country_names:
#     print(f'Searching {country}...')
    for n in range(1, 10):
        try:
            url_base = f'https://theblondeabroad.com/category/{country}/page/{n}/'
            res_ = requests.get(url_base)
            soup_ = BeautifulSoup(res_.content, 'lxml')
            time.sleep(1)

            for link in soup_.find_all(attrs={'class': 'entry-title-link'}):
                article = {}
                article['href'] = link.attrs['href']
                article['title'] = link.text.strip()
                article['country'] = country
                articles.append(article)
        except:
            print(f'Error with {country} at page {n}.')

df = pd.DataFrame(articles)
# df.to_csv('./data/all_articles.csv')
print('Shape of dataframe:', df.shape)
df.head()

Shape of dataframe: (536, 3)


,href,title,country
0,https://theblondeabroad.com/best-time-to-trave...,The Best Time to Travel to Austria,Austria
1,https://theblondeabroad.com/20-photos-to-inspi...,20 Photos to Inspire You to Visit the Christma...,Austria
2,https://theblondeabroad.com/visiting-the-chris...,Visiting the Christmas Markets in Vienna,Austria
3,https://theblondeabroad.com/danube-christmas-m...,Danube Christmas Market River Cruise with Crys...,Austria
4,https://theblondeabroad.com/river-cruise-onboa...,River Cruise Onboard Crystal Cruises’ Mozart,Austria


In [6]:
content_filed = []
for url in df['href']:
    res_post = requests.get(url)
    soup_post = BeautifulSoup(res_post.content, 'lxml')

    paragraphs = soup_post.find_all(attrs={'class': 'entry-content'})

    text = ' '.join([paragraph.text.replace('\n', '') for paragraph in paragraphs])
    content_filed.append(text)
    
    time.sleep(1)
df['text'] = content_filed

In [7]:
df.tail()

,href,title,country,text
531,https://theblondeabroad.com/what-to-pack-for-a...,What to Pack for an African Safari,Zambia,Updated: October 2019An African safari was one...
532,https://theblondeabroad.com/microlight-flight-...,Microlight Flight Over Victoria Falls,Zimbabwe,Victoria Falls had been one of my dream destin...
533,https://theblondeabroad.com/visiting-victoria-...,Guide to Visiting Victoria Falls from Zimbabwe...,Zimbabwe,Victoria Falls was a dream destination of mine...
534,https://theblondeabroad.com/staying-at-tongabe...,Staying at Tongabezi Lodge near Victoria Falls,Zimbabwe,I’ve stayed at a lot of amazing places around ...
535,https://theblondeabroad.com/what-to-pack-for-a...,What to Pack for an African Safari,Zimbabwe,Updated: October 2019An African safari was one...


In [8]:
df.to_csv('./data/articles_theblondeabroad.csv', index=False)